# Loading the HST data

### Introduction

Coming

## The code

The code written below uses functions in ```src/calculations.py```. It is capable of looking for variations in the entire dataset (e.g. it is capable of running over each HST visit).

### Import the standard routines and load parameters

In [1]:
import numpy as np
import json, sys, os

import matplotlib.pyplot as plt
from matplotlib import rc, cm

# get the path of the current directory
path = os.getcwd()
home = os.path.dirname(path)

# Print the repository home directory
print("Repository home directory:",home)

# Add the src folder to the system path
sys.path.append(home+'/src')

# Import the python functions from src
from calculations import Calc, Model, Stats

# We shorten the functions name to make it easier to call the required a functions
c   = Calc()
m   = Model()
s   = Stats()

Repository home directory: /home/pas/science/exocomets


In [2]:
# Importing parameters from a json file.
with open(home+'/params.json') as param_file:    
   param = json.load(param_file)

In [3]:
# We read in the data directories we are using. To see what directories this is open params.json.
datadirs = param["datadirs"]

# We select part A which is the red end of the spectrum (the other part being B, which is the blue end)
part     = param["BetaPictoris"]["part"]

### Get the data

In [ ]:
D = []

print("Data used for this analysis:")

for i in sorted(datadirs):
    if param["filenames"]["split_files"] == 'yes':
        if datadirs[i][5:-1] in ["2017-04-23","2017-06-10","2017-08-04","2017-10-21","2017-11-26","2018-03-17","2018-05-09"]:
            print("\n",datadirs[i])
            D.append(c.GetData(param, home+'/'+datadirs[i]))
    else:
        print("\n",datadirs[i][5:-1])
        D.append(c.GetData(param, home+'/'+datadirs[i]))
    
# We save the data
np.savez(home+'/data/D_'+part+'.npz', D, dtype=object)

Data used for this analysis:

 data/2017-04-23/
	AVM shift: 0.0" 	EXP: 526s, 	Date: 2017-04-22, Time: 23:36:06 UTC
	AVM shift: 0.0" 	EXP: 526s, 	Date: 2017-04-22, Time: 23:48:03 UTC
	AVM shift: 0.0" 	EXP: 526s, 	Date: 2017-04-23, Time: 00:00:00 UTC
	AVM shift: 0.0" 	EXP: 526s, 	Date: 2017-04-23, Time: 00:11:57 UTC
	AVM shift: -1.25" 	EXP: 664s, 	Date: 2017-04-23, Time: 00:31:55 UTC
	AVM shift: -1.25" 	EXP: 664s, 	Date: 2017-04-23, Time: 00:45:32 UTC
	AVM shift: -1.25" 	EXP: 664s, 	Date: 2017-04-23, Time: 00:59:09 UTC
	AVM shift: -1.25" 	EXP: 664s, 	Date: 2017-04-23, Time: 01:12:46 UTC
	AVM shift: 1.1" 	EXP: 664s, 	Date: 2017-04-23, Time: 02:07:24 UTC
	AVM shift: 1.1" 	EXP: 664s, 	Date: 2017-04-23, Time: 02:21:01 UTC
	AVM shift: 1.1" 	EXP: 664s, 	Date: 2017-04-23, Time: 02:34:38 UTC
	AVM shift: 1.1" 	EXP: 664s, 	Date: 2017-04-23, Time: 02:48:15 UTC

 data/2017-06-10/
	AVM shift: 0.0" 	EXP: 526s, 	Date: 2017-06-10, Time: 20:02:24 UTC
	AVM shift: 0.0" 	EXP: 526s, 	Date: 2017-06-10, Time: 

### We now normalise, shift and re-normalise the data 

In [ ]:
# Load the data into the multidimentional array D
D = c.LoadData(home+'/data/D_'+part+'.npz')

# Normalise the data
Dn = c.NormSpec(param, D)

# Shift the data relative to the first spectrum of each visit
Dns = c.ShiftSpec(param, Dn)

# Re normalise the data
Dnsn = c.NormSpec(param, Dns)

# We save the normalised and shifted data
np.savez(home+'/data/Dnsn_'+part+'.npz', Dns, dtype=object)